# Pink Floyd Lyrics

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [32]:
file = "../raw_data/pink_floyd_lyrics.csv"
pink_df = pd.read_csv(file, decimal=",")

In [33]:
pink_df

,album,song_title,year,lyrics
0,The Piper at the Gates of Dawn,Astronomy Domine,1967-08-05,"""Moon in both [houses]...""...Scorpio, [Arabian..."
1,The Piper at the Gates of Dawn,Lucifer Sam,1967-08-05,"Lucifer Sam, siam cat\nAlways sitting by your ..."
2,The Piper at the Gates of Dawn,Matilda Mother,1967-08-05,There was a king who ruled the land\nHis Majes...
3,The Piper at the Gates of Dawn,Flaming,1967-08-05,Alone in the clouds all blue\nLying on an eide...
4,The Piper at the Gates of Dawn,Pow R. Toc H.,1967-08-05,TCH TCH\nAHH (AHH)\nTCH TCH\nAHH AHH\nDoi doi\...
...,...,...,...,...
158,The Endless River,Surfacing,2014-11-07,NaN
159,The Endless River,Louder than Words,2014-10-14,We bitch and we fight\nDiss each other on sigh...
160,The Endless River,TBS9,2014-11-07,NaN
161,The Endless River,TBS14,2014-11-07,NaN


In [34]:
pink_df = pink_df.replace(r'\n',' ', regex=True) 

In [35]:
pink_df['year'] = pd.to_datetime(pink_df.year)

In [36]:
pink_df.dropna(subset = ["lyrics"], inplace=True)


In [37]:
pink_df['lyrics'] = pink_df['lyrics'].str.lower()


In [38]:
pink_df['lyrics']= pink_df['lyrics'].str.replace('[^A-Za-z\s]+', ' ')


/var/folders/86/b58mx9y52mq_z0wkgsjtgjqh0000gn/T/ipykernel_7612/1971328018.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  pink_df['lyrics']= pink_df['lyrics'].str.replace('[^A-Za-z\s]+', ' ')


In [56]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def get_mock_up_data():
    

    X = pink_df['lyrics']
    y = np.array([1., 0., 0.])

    ### Let's tokenize the vocabulary 
    tk = Tokenizer()
    tk.fit_on_texts(X)
    vocab_size = len(tk.word_index)
    print(f'There are {vocab_size} different words in your corpus')
    X_token = tk.texts_to_sequences(X)

    ### Pad the inputs
    X_pad = pad_sequences(X_token, dtype='float32', padding='post')
    
    return X_pad, y, vocab_size

X_pad, y, vocab_size = get_mock_up_data()
print("X_pad.shape", X_pad.shape)
X_pad


There are 2846 different words in your corpus
X_pad.shape (125, 439)


array([[ 207.,    8.,  394., ...,    0.,    0.,    0.],
       [ 892., 1353., 1354., ...,    0.,    0.,    0.],
       [  27.,   25.,    5., ...,    0.,    0.,    0.],
       ...,
       [  16., 1103.,   37., ...,    0.,    0.,    0.],
       [2823.,  172., 2824., ...,    0.,    0.,    0.],
       [  16., 2836.,    3., ...,    0.,    0.,    0.]], dtype=float32)

In [53]:
#px.scatter(pink_df['year'], pink_df['song_title'], color= pink_df['album'])

In [57]:
### Let's build the neural network now
from tensorflow.keras import layers, Sequential

# Size of your embedding space = size of the vector representing each word
embedding_size = 100

model = Sequential()
model.add(layers.Embedding(
    input_dim=vocab_size+1, # 16 +1 for the 0 padding
    input_length=8, # Max_sentence_length (optional, for model summary)
    output_dim=embedding_size, # 100
    mask_zero=True, # Built-in masking layer :)
))

model.add(layers.LSTM(20))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

2022-06-28 14:47:14.298407: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 8, 100)            284700    
                                                                 
 lstm (LSTM)                 (None, 20)                9680      
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 294,401
Trainable params: 294,401
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit(X_pad, y, epochs=5, batch_size=16, verbose=0)

ValueError: Data cardinality is ambiguous:
  x sizes: 125
  y sizes: 3
Make sure all arrays contain the same number of samples.